# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

# 1) Load data & review the data

How many rows are there in the "titanic_to_student.csv"?

In [2]:
# Import the libraries
import numpy as np
import pandas as pd

In [34]:
df = pd.read_csv("titanic_to_student.csv", index_col=0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
1,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
2,6,0.0,3.0,"Moran, Mr. James",male,NaN,0.0,0,330877,8.4583,NaN,Q
3,8,0.0,3.0,"Palsson, Master. Gosta Leonard",male,2.0,3.0,1,349909,21.0750,NaN,S
4,10,1.0,2.0,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1.0,0,237736,30.0708,NaN,C


In [64]:
df.shape[0]

445

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 445 entries, 0 to 444
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  445 non-null    int64  
 1   Survived     432 non-null    float64
 2   Pclass       414 non-null    float64
 3   Name         433 non-null    object 
 4   Sex          445 non-null    object 
 5   Age          360 non-null    float64
 6   SibSp        428 non-null    float64
 7   Parch        445 non-null    int64  
 8   Ticket       426 non-null    object 
 9   Fare         445 non-null    float64
 10  Cabin        116 non-null    object 
 11  Embarked     400 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 45.2+ KB


In [15]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,445.000000,432.000000,414.000000,360.000000,428.000000,445.000000,445.000000
mean,446.000000,0.416667,2.265700,29.136361,0.516355,0.379775,34.238473
std,257.209383,0.493578,0.853384,14.285739,1.083067,0.795282,52.825157
min,2.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.000000,0.000000,1.000000,20.000000,0.000000,0.000000,7.925000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,15.245800
75%,668.000000,1.000000,3.000000,37.250000,1.000000,0.000000,34.375000
max,890.000000,1.000000,3.000000,74.000000,8.000000,5.000000,512.329200


# 2) Drop unqualified variables
2.1 Drop variables with missing > 50%

2.2 Check all columns except 'Age' and 'Fare' for flat values, drop the columns where flat value > 70%
From 2.1 and 2.2, how many columns do we have left?

Note: 
    
    -Ensure missing values are considered in your calculation. If you use normalize in .value_counts(), please include dropna=False.






In [48]:
missing_threshold = 0.5 * df.shape[0]
non_flat_column = ["Age", "Fare"]
cols_to_drop = df.columns[df.isna().sum() > missing_threshold]

for i in df.columns:
    if i in non_flat_column or i in cols_to_drop:
        continue
    most_freq = df[i].value_counts(normalize=True, dropna=False).iloc[0]
    if most_freq > 0.7:
        cols_to_drop = cols_to_drop.union(pd.Index([i]))

df.drop(cols_to_drop.to_list(), axis=1, inplace=True)
df.shape[1]

10

# 3) Remove all rows with missing target (the variable "Survived")

Remove all rows with missing targets (the variable "Survived")

How many rows do we have left?


In [51]:
df.dropna(subset=["Survived"], inplace=True)
df.shape[0]

432

# 4) Handle outliers 
Handle outliers

For the variable “Fare”, replace outlier values with the boundary values

If value < (Q1 - 1.5IQR), replace with (Q1 - 1.5IQR)

If value > (Q3 + 1.5IQR), replace with (Q3 + 1.5IQR)

What is the average (mean) of “Fare” after replacing the outliers (round 2 decimal points)?

Hint: Use function round(_, 2)



In [65]:
q1 = df["Fare"].quantile(0.25)
q3 = df["Fare"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
df["Fare"] = df["Fare"].apply(
    lambda x: (
        lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x)
    )
)
round(df["Fare"].mean(), 2)

np.float64(26.27)

# 5) Impute missing value

Impute missing value

For number type column, impute missing values with mean

What is the average (mean) of “Age” after imputing the missing values (round 2 decimal points)?

Hint: Use function round(_, 2)


In [74]:
df[df["Age"].isna()]
df["Age"].mean()

np.float64(29.13636111111111)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
df["Age"] = pd.DataFrame(imputer.fit_transform(df[["Age"]]))

round(df['Age'].mean(), 2)

np.float64(29.136361111111107)

# 6) Convert categorical to numeric values

Convert categorical to numeric values

For the variable “Embarked”, perform the dummy coding.

What is the average (mean) of “Embarked_Q” after performing dummy coding (round 2 decimal points)?

Hint: Use function round(_, 2)



In [24]:
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder(handle_unknown="error", drop="first")
encoded_embarked = enc.fit_transform(df[["Embarked"]]).toarray()
encoded_column = pd.DataFrame(
    columns=enc.get_feature_names_out(["Embarked"]), data=encoded_embarked
)
df = pd.concat(
    [df.reset_index(drop=True), encoded_column.reset_index(drop=True)], axis=1
)
round(df["Embarked_Q"].mean(), 2)

np.float64(0.06)

# 7) Partition data
Split train/test split with stratification using 70%:30% and random seed with 123

Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

What is the proportion of survivors (survived = 1) in the training data (round 2 decimal points)?

Hint: Use function round(_, 2), and train_test_split() from sklearn.model_selection




In [53]:
df = pd.read_csv("titanic_to_student.csv", index_col=0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
1,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
2,6,0.0,3.0,"Moran, Mr. James",male,NaN,0.0,0,330877,8.4583,NaN,Q
3,8,0.0,3.0,"Palsson, Master. Gosta Leonard",male,2.0,3.0,1,349909,21.0750,NaN,S
4,10,1.0,2.0,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1.0,0,237736,30.0708,NaN,C


In [54]:
from sklearn.model_selection import train_test_split

df.fillna(df.select_dtypes(include="number").mean(), inplace=True)
target = df.pop("Survived")
x_train, x_test, y_train, y_test = train_test_split(
    df, target, train_size=0.7, stratify=target, random_state=123
)

round((pd.Series)(y_train).eq(1).mean(), 2)

np.float64(0.41)